## Install TDW

In [1]:
!pip install tdw

## Import Packages

In [1]:
from tdw.controller import Controller
from tdw.add_ons.oculus_touch import OculusTouch
from tdw.output_data import OutputData
from tdw.add_ons.third_person_camera import ThirdPersonCamera
from tdw.tdw_utils import TDWUtils
from tdw.librarian import ModelLibrarian
from tdw.output_data import OutputData, Bounds
from scipy.spatial.transform import Rotation as R
from tdw.add_ons.image_capture import ImageCapture
from tdw.librarian import ModelLibrarian
import numpy as np
import os
import time

## Create Libraries/Folder to save recorded files
IMPORTANT: If you are using Mac, replace "Windows" with "Darwin" in the 6th line and if you are using Linux, replace it with "Linux"

In [2]:
current_dir = os.path.dirname(os.getcwd())
if os.path.exists(os.path.join(current_dir, "assets")):
    local_librarian = ModelLibrarian(
        library=os.path.join(current_dir, "assets", "library.json")
    )
    models_directory = (
        os.path.join(current_dir, "assets", "Windows").replace("\\", "/") + "/"
    )
else:
    print("No local models found")
    models_directory = None
    local_librarian = None

images_dir = os.path.join(current_dir, "images")
if not os.path.exists(images_dir):
    os.makedirs(images_dir)

videos_dir = os.path.join(current_dir, "videos")
if not os.path.exists(videos_dir):
    os.makedirs(videos_dir)

No local models found


## Helper functions

In [3]:
def get_bounds(c, obj_id, rotation):
    resp = c.communicate({"$type": "send_bounds", "frequency": "once"})
    for i in range(len(resp) - 1):
        r_id = OutputData.get_data_type_id(resp[i])
        if r_id == "boun":
            bounds = Bounds(resp[i])
            for j in range(bounds.get_num()):
                if bounds.get_id(j) == obj_id:
                    top = bounds.get_top(j)[1]
                    bottom = bounds.get_bottom(j)[1]
                    back = bounds.get_back(j)[2]
                    front = bounds.get_front(j)[2]
                    left = bounds.get_left(j)[0]
                    right = bounds.get_right(j)[0]
    init_corners = np.array(
        [
            [left, top, back],
            [left, top, front],
            [left, bottom, back],
            [left, bottom, front],
            [right, top, back],
            [right, top, front],
            [right, bottom, back],
            [right, bottom, front],
        ]
    )

    r = (
        R.from_euler(
            "xyz", [[rotation["x"], rotation["y"], rotation["z"]]], degrees=True
        )
        .as_matrix()
        .squeeze()
    )

    transformed_corners = np.matmul(r, init_corners.T).T

    return {
        "left": float(left),
        "right": float(right),
        "top": float(top),
        "bottom": float(bottom),
        "front": float(front),
        "back": float(back),
        "width": abs(float(left - right)),
        "height": abs(float(top - bottom)),
        "depth": abs(float(front - back)),
        "corners": transformed_corners.tolist(),
    }


def add_your_own_object(
    c,
    name,
    id,
    position,
    rotation={"x": 0, "y": 0, "z": 0},
    mass=5,
    color=None,
    scale={"x": 1, "y": 1, "z": 1},
    material=None,
    dynamic_friction=0.3,
    static_friction=0.3,
    bounciness=0.7,
    set_kinematic=False,
    use_gravity=True
):
    commands = []
    commands.extend(
        [
            {
                "$type": "add_object",
                "name": name,
                "url": "file:///" + models_directory + name,
                "id": id,
                "position": position,
            },
            {
                "$type": "scale_object",
                "scale_factor": scale,
                "id": id,
            },
        ]
    )
    if material:
        model_record = local_librarian.get_record(name)
        commands.extend(
            TDWUtils.set_visual_material(
                c=c,
                substructure=model_record.substructure,
                material=material,
                object_id=id,
            )
        )
    commands.append({"$type": "set_mass", "mass": mass, "id": id})
    commands.extend(
        [
            {
                "$type": "rotate_object_by",
                "angle": rotation["x"],
                "id": id,
                "axis": "pitch",
                "use_centroid": True,
            },
            {
                "$type": "rotate_object_by",
                "angle": rotation["y"],
                "id": id,
                "axis": "yaw",
                "use_centroid": True,
            },
            {
                "$type": "rotate_object_by",
                "angle": rotation["z"],
                "id": id,
                "axis": "roll",
                "use_centroid": True,
            },
        ]
    )

    commands.extend(
        [
            {
                "$type": "set_kinematic_state",
                "id": id, 
                "is_kinematic": set_kinematic, 
                "use_gravity": use_gravity
            },
            {
                "$type": "set_physic_material",
                "dynamic_friction": dynamic_friction, 
                "static_friction": static_friction, 
                "bounciness": bounciness, 
                "id": id
            }
        ]
    )

    if color:
        commands.append({"$type": "set_color", "color": color, "id": id})
    c.communicate(commands)
    bounds = get_bounds(c, id, rotation)
    c.communicate([])

    return bounds

def add_tdw_object(
    c,
    name,
    id,
    position,
    rotation={"x": 0, "y": 0, "z": 0},
    mass=5,
    color=None,
    scale={"x": 1, "y": 1, "z": 1},
    material=None,
    dynamic_friction=0.3,
    static_friction=0.3,
    bounciness=0.7,
    set_kinematic=False,
    use_gravity=True,
    library="models_core.json"
):
    commands = []
    commands.extend(
        c.get_add_physics_object(
            model_name=name,
            object_id=id,
            position=position,
            rotation=rotation,
            mass=mass,
            default_physics_values = False,
            dynamic_friction=dynamic_friction,
            static_friction=static_friction,
            bounciness=bounciness,
            gravity=use_gravity,
            kinematic=set_kinematic,
            scale_factor=scale,
            library=library
        )
    )

    if color:
        commands.append({"$type": "set_color", "color": color, "id": id})

    if material:
        librarian = ModelLibrarian(library=library)
        model_record = librarian.get_record(name)
        commands.extend(
            TDWUtils.set_visual_material(
                c=c,
                substructure=model_record.substructure,
                material=material,
                object_id=id,
            )
        )

    c.communicate(commands)
    bounds = get_bounds(c, id, rotation)
    
    return bounds

## Controller
A controller is the Python object that communicates with the simulation application (the build). You can code up your scene in the controller and send it to the simulator for rendering. Similarly, the simulator sends the output data (including pose of every object and agent, collisions, RGB and depth maps, etc) back to the controller and you can read them out.

In [4]:
c = Controller(launch_build=False)

You are using TDW 1.12.18.0 but version 1.13.0.0 is available.
Consider upgrading:
pip3 install tdw -U
Build version 1.12.18
Unity Engine 2020.3.24f1
Python tdw module version 1.12.18.0


## Create a scene
You need a room where stuff happen in. Here we use one of the many rooms that exist in TDW. To see other rooms, take a look at:

https://github.com/threedworld-mit/tdw/blob/master/Documentation/lessons/core_concepts/scenes.md

This step is going to take some time to run....

In [5]:
c.communicate(c.get_add_scene(scene_name="tdw_room"))
c.communicate(
    [
        TDWUtils.create_empty_room(15, 15),
        {"$type": "set_screen_size", "width": 1000, "height": 1000},
        {"$type": "set_target_framerate", "framerate": 60},
    ]
)

[b'\x00\x00\x00\x03']

You may wonder why you don't see anything. That's because you need a camera to act as your eyes:

In [6]:
camera_1 = ThirdPersonCamera(avatar_id="a",
                           position={"x": 0, "y": 1.6, "z": -1.0},
                           rotation={"x": 0, "y": 0, "z": 0},
                           field_of_view=105
)
c.add_ons.append(camera_1)
c.communicate([])

[b'\x00\x00\x00\x04']

Now it's time to add objects. TDW has thousands of existing objects in three libraries. You can use the command below to see a list of all existing objects:

In [7]:
libraries = ["models_core.json", "models_special.json", "models_flex.json"]
for library in libraries:
    librarian = ModelLibrarian(library)
    for record in librarian.records:
        print(library, record.name)

models_core.json 034_vray
models_core.json 102_pepsi_can_12_fl_oz_vray
models_core.json 104_sprite_can_12_fl_oz_vray
models_core.json 12_06_001
models_core.json 24_in_wall_cabinet_white_wood
models_core.json 24_in_wall_cabinet_wood_beach_honey
models_core.json 36_in_wall_cabinet_white_wood
models_core.json 36_in_wall_cabinet_wood_beach_honey
models_core.json 4ft_shelf_metal
models_core.json 4ft_wood_shelving
models_core.json 5ft_shelf_metal
models_core.json 5ft_wood_shelving
models_core.json 699264_shoppingcart_2013
models_core.json 6ft_shelf_metal
models_core.json 6ft_wood_shelving
models_core.json 868580_pliers_max2016
models_core.json 9v_battery
models_core.json aaa_battery
models_core.json afl_lamp
models_core.json alarm_clock
models_core.json alivar_tech_bench_sofa
models_core.json alma_floor_lamp
models_core.json amphora_jar_vase
models_core.json apple
models_core.json apple_ipod_touch_grey_vray
models_core.json apple_ipod_touch_pink_vray
models_core.json apple_ipod_touch_yellow_

Now let's add some objects to the scene, starting with a table:

In [11]:
table_stats = {
    "name" : "small_table_green_marble",
    "id" : c.get_unique_id(),
    "position" : {"x" : 0.0, "y" : 0.0, "z" : 0},
    "rotation" : {"x" : 0, "y" : 0, "z" : 0},
    "scale" : {"x" : 1.5, "y" : 1.2, "z" : 1.5},
    "color" : None,
    "material" : None,
    "mass" : 2000,
    "dynamic_friction" : 0.45,
    "static_friction" : 0.48,
    "bounciness" : 0.0,
    "set_kinematic" : False,
    "use_gravity" : True
}
table_bounds = add_tdw_object(c, **table_stats)

Take a moment to look at the output you get. It has the boundary information of the object you added, including its width, height, depth, x value of the left and right corners, y value of the top and bottom corners, z value of the front and back corners, and the coordinations of all corners

In [9]:
table_bounds

{'left': -0.8583490252494812,
 'right': 0.857244074344635,
 'top': 1.0799124240875244,
 'bottom': 0.008413001894950867,
 'front': 0.568520724773407,
 'back': -0.5684612393379211,
 'width': 1.7155930995941162,
 'height': 1.0714994668960571,
 'depth': 1.1369819641113281,
 'corners': [[-0.8583490252494812, 1.0799124240875244, -0.5684612393379211],
  [-0.8583490252494812, 1.0799124240875244, 0.568520724773407],
  [-0.8583490252494812, 0.008413001894950867, -0.5684612393379211],
  [-0.8583490252494812, 0.008413001894950867, 0.568520724773407],
  [0.857244074344635, 1.0799124240875244, -0.5684612393379211],
  [0.857244074344635, 1.0799124240875244, 0.568520724773407],
  [0.857244074344635, 0.008413001894950867, -0.5684612393379211],
  [0.857244074344635, 0.008413001894950867, 0.568520724773407]]}

You can remove the table using the command below (uncomment and run it to see how it works then run the cell above again before continueing):

In [10]:
# c.communicate([
#     {"$type": "destroy_object", "id": table_stats["id"]},
#     {"$type": "send_rigidbodies", "frequency": "never"}
# ])

[b'\x00\x00\x00\x07']

You can also teleport the object and rotate it after adding

In [13]:
c.communicate(
    [    
        {
            "$type": "teleport_object",
            "position": {"x": 0, "y": 1, "z": 0},
            "id": table_stats["id"]
        },
        {
            "$type": "rotate_object_to",
            "rotation": {"w": 1, "x": 0, "y": 0, "z": 0},
            "id": table_stats["id"]
        }
    ]
)

[b'\x00\x00\x00\n']

If you set the position such that the table ends up in the air, you see if does not fall down. That's because the simulation is not running. You can use the command below to run the simulation for a certain number of frames:

In [14]:
n_frames = 50
for i in range(n_frames):
    c.communicate([])

Now let's add more objects. We can start by placing a box on top of the table. You can extract the y value of table top from table_bounds["top"]  

In [15]:
station_stats = {
    "name" : "iron_box",
    "id" : c.get_unique_id(),
    "position" : {"x" : 0, "y" : table_bounds["top"], "z" : 0.15},
    "rotation" : {"x" : 0, "y" : 0, "z" : 0},
    "scale" : {"x" : 2.0, "y" : 0.7, "z" : 4.0},
    "color" : None,
    "material" : "parquet_wood_red_cedar",
    "mass" : 1000,
    "dynamic_friction" : 0.15,
    "static_friction" : 0.18,
    "bounciness" : 0.0,
    "set_kinematic" : False,
    "use_gravity" : True,
}
station_bounds = add_tdw_object(c, **station_stats)

Let's add a ramp on top

In [16]:
ramp_stats = {
    "name" : "ramp_with_platform_weld",
    "id" : c.get_unique_id(),
    "position" : {"x" : 0, "y" : station_bounds["top"], "z" : 0.15},
    "rotation" : {"x" : 0, "y" : -90, "z" : 0},
    "scale" : {"x" : 0.2, "y" : 0.4, "z" : 0.4},
    "color" : {"r" : 0.7, "g" : 0.7, "b" : 1},
    "mass" : 1000,
    "dynamic_friction" : 0.15,
    "static_friction" : 0.18,
    "bounciness" : 0.8,
    "set_kinematic" : False,
    "use_gravity" : True,
    "library" : "models_full.json"
}
ramp_bounds = add_tdw_object(c, **ramp_stats)

Now let's place walls on the left and right side (we don't want any object that slides to fall from the side)

In [17]:
left_wall_stats = {
    "name" : "iron_box",
    "id" : c.get_unique_id(),
    "position" : {"x" : -0.42, "y" : table_bounds["top"], "z" : 0.05},
    "rotation" : {"x" : 0, "y" : 0, "z" : 0},
    "scale" : {"x" : 0.1, "y" : 2.5, "z" : 3.0},
    "color" : None,
    "material" : "parquet_wood_red_cedar",
    "mass" : 1000,
    "dynamic_friction" : 0.15,
    "static_friction" : 0.18,
    "bounciness" : 0.8,
    "set_kinematic" : False,
    "use_gravity" : True
}
left_wall_bounds = add_tdw_object(c, **left_wall_stats)

right_wall_stats = {
    "name" : "iron_box",
    "id" : c.get_unique_id(),
    "position" : {"x" : 0.42, "y" : table_bounds["top"], "z" : 0.05},
    "rotation" : {"x" : 0, "y" : 0, "z" : 0},
    "scale" : {"x" : 0.1, "y" : 2.5, "z" : 3.0},
    "color" : None,
    "material" : "parquet_wood_red_cedar",
    "mass" : 1000,
    "dynamic_friction" : 0.15,
    "static_friction" : 0.18,
    "bounciness" : 0.8,
    "set_kinematic" : False,
    "use_gravity" : True
}
right_wall_bounds = add_tdw_object(c, **right_wall_stats)

Now let's place a ball on top of the ramp. The ball is going to slide down yet so we don't run the simulation

In [18]:
ball_stats = {
    "name" : "prim_sphere",
    "id" : c.get_unique_id(),
    "position" : {"x" : 0, "y" : ramp_bounds["top"] + 0.1, "z" : 0.25},
    "rotation" : {"x" : 0, "y" : 0, "z" : 0},
    "scale" : {"x" : 0.06, "y" : 0.06, "z" : 0.06},
    "color" : {"r" : 0.8, "g" : 0, "b" : 0},
    "mass" : 2.0,
    "dynamic_friction" : 0.15,
    "static_friction" : 0.18,
    "bounciness" : 0.8,
    "set_kinematic" : False,
    "use_gravity" : True,
    "library" : "models_special.json"
}
ball_bounds = add_tdw_object(c, **ball_stats)

At this point, you can also attach a recorder to your camera to capture the video of the motion

In [19]:
capture = ImageCapture(path=images_dir, avatar_ids=["a"])
c.add_ons.append(capture)

But it's no fun if the ball just falls down. We all know how it's gonna move. It will be more fun if a force with random direction and amplitude is applied to the ball.

In [20]:
force_vector = {"x": np.random.uniform(-0.001, 0.001), "y": 0, "z": np.random.uniform(-0.0005, 0)}
print("force vector: {}".format(force_vector))
c.communicate(
    {
        "$type": "apply_force_to_object",
        "id": ball_stats["id"],
        "force": force_vector,
    }
)

# Run the simulation for 3 seconds
start_time = time.time()
while time.time() - start_time < 3:
    c.communicate([])

force vector: {'x': -0.00027734723652325904, 'y': 0, 'z': -0.0003517335731325273}


Take a look into the images folder. You can see a picture of every frame saved in the folder. If you want to attch frames to each other to make a video, check out

https://github.com/threedworld-mit/tdw/blob/master/Documentation/lessons/video/overview.md

Finally, run the command below to terminate the simulation

In [21]:
c.communicate({"$type": "terminate"})

[b'\x10\x00\x00\x00quit\x00\x00\x06\x00\x08\x00\x07\x00\x06\x00\x00\x00\x00\x00\x00\x01',
 b'\x18\x00\x00\x00imag\x00\x00\x0e\x00\x18\x00\x04\x00\x08\x00\x0c\x00\x10\x00\x14\x00\x0e\x00\x00\x00(\x00\x00\x00\x10\x00\x00\x00\xe8\x03\x00\x00\xe8\x03\x00\x00 \x00\x00\x00\x0f\x00\x00\x00SensorContainer\x00\x01\x00\x00\x00a\x00\x00\x00\x01\x00\x00\x00\x10\x00\x00\x00\x00\x00\n\x00\x0c\x00\x00\x00\x08\x00\x07\x00\n\x00\x00\x00\x00\x00\x00\x02\x04\x00\x00\x00\xcb>\x01\x00\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x02\x00\x00\x01\x00\x01\x00\x00\xff\xc0\x00\x11\x08\x03\xe8\x03\xe8\x03\x00"\x00\x01\x11\x01\x02\x11\x01\xff\xdb\x00C\x00\x08\x06\x06\x07\x06\x05\x08\x07\x07\x07\t\t\x08\n\x0c\x14\r\x0c\x0b\x0b\x0c\x19\x12\x13\x0f\x14\x1d\x1a\x1f\x1e\x1d\x1a\x1c\x1c $.\' ",#\x1c\x1c(7),01444\x1f\'9=82<.342\xff\xdb\x00C\x01\t\t\t\x0c\x0b\x0c\x18\r\r\x182!\x1c!22222222222222222222222222222222222222222222222222\xff\xc4\x00\x1f\x00\x00\x01\x05\x01\x01\x01\x01\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x01\x02\x03\

The ball is going to roll down as soon as we run the simulation. But what do we want to do with the ball? One option is to show them a video of the ball rolling down half way and ask them to click where they think the ball is going to hit the table. 